<a href="https://colab.research.google.com/github/Sakinat-Folorunso/CMP_805_Advanced_Programming_Languages/blob/main/notebooks/CMP805_Week5_PH_Python_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMP805 — Week 5 Practical (Python, Colab)
**Topic:** Polymorphism & Inference — Hindley–Milner (Algorithm W) with `let`‑polymorphism; intro to variance  
**Course:** Advanced Programming Languages (M.Sc.), OOU — CMP805

**Instructor:** **DR SAKINAT FOLORUNSO – ASSOCIATE PROFESSOR OF AI SYSTEMS AND FAIR DATA**  
**Department:** **COMPUTER SCIENCES, OLABISI ONABANJO UNIVERSITY, AGO‑IWOYE, OGUN STATE, NIGERIA**

> This PH implements a small **HM type inferencer** (no annotations) over a λ‑calculus with `let`, `if`, `+`, `==`, and pairs. It aligns with Week‑5 in your outline (HM, unification/constraints, subtyping/variance exercises).

### Learning goals (≈60 minutes)
- Implement **Algorithm W**: unification, substitutions, instantiation/generalization, and `let`‑polymorphism.
- Infer principal types for programs without annotations.
- (Conceptual) Reason about **variance** (co/contra‑variance of `→`) via short exercises.

In [ ]:
# 🧑‍🎓 Student info
STUDENT_NAME = "Type your full name here"
STUDENT_ID   = "Matric/ID here"
print("Student:", STUDENT_NAME, "| ID:", STUDENT_ID)

In [ ]:
# ✅ Environment check
import sys
major, minor = sys.version_info[:2]
assert (major, minor) >= (3, 10), f"Need Python 3.10+, found {major}.{minor}"
print(f"Python {major}.{minor} OK — match/case available.")

In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, Optional, Set, Tuple, List

# =============================
# 🧱 Types and type schemes
# =============================
@dataclass(frozen=True)
class TVar:                      # Type variable, e.g., a, b, c
    id: int

@dataclass(frozen=True)
class TCon:                      # Base type constructor, e.g., Int, Bool
    name: str

@dataclass(frozen=True)
class TFun:                      # Function type τ1 -> τ2
    arg: "Type"
    ret: "Type"

@dataclass(frozen=True)
class TPair:                     # Pair type τ1 * τ2
    fst: "Type"
    snd: "Type"

Type = TVar | TCon | TFun | TPair

INT  = TCon("Int")              # Built-in base types
BOOL = TCon("Bool")

@dataclass(frozen=True)
class Scheme:                    # ∀α. τ  (universally quantified type)
    vars: frozenset[int]
    ty: Type

In [ ]:
# =============================
# 🔤 Pretty-printing of types
# =============================
from collections import OrderedDict

def _collect_tvars(t: Type, acc: Set[int] | None = None) -> Set[int]:
    if acc is None: acc = set()
    if isinstance(t, TVar):
        acc.add(t.id)
    elif isinstance(t, TFun):
        _collect_tvars(t.arg, acc); _collect_tvars(t.ret, acc)
    elif isinstance(t, TPair):
        _collect_tvars(t.fst, acc); _collect_tvars(t.snd, acc)
    # TCon has no vars
    return acc

def _pp_map(t: Type) -> Dict[int, str]:
    # Map type variable IDs to letters a, b, c,...
    ids = list(_collect_tvars(t))
    letters = "abcdefghijklmnopqrstuvwxyz"
    mapping = {}
    for i, tv in enumerate(ids):
        mapping[tv] = letters[i % 26] + ("" if i < 26 else str(i // 26))
    return mapping

def show_type(t: Type, envmap: Dict[int, str] | None = None) -> str:
    m = envmap or _pp_map(t)
    def go(x: Type) -> str:
        if isinstance(x, TCon): return x.name
        if isinstance(x, TVar): return m.get(x.id, f"t{x.id}")
        if isinstance(x, TPair):
            return f"{go(x.fst)} * {go(x.snd)}"
        if isinstance(x, TFun):
            left = go(x.arg)
            right = go(x.ret)
            if isinstance(x.arg, TFun):
                left = f"({left})"
            return f"{left} -> {right}"
        raise TypeError("unknown type")
    return go(t)

def show_scheme(s: Scheme) -> str:
    if not s.vars: return show_type(s.ty)
    # Ensure bound vars come first in printing order
    m = {vid: chr(ord('a')+i) for i, vid in enumerate(sorted(s.vars))}
    return f"forall {' '.join(m[v] for v in sorted(s.vars))}. {show_type(s.ty, m)}"

In [ ]:
# =================================
# 🔧 Substitutions, FTV, and apply
# =================================
Subst = Dict[int, Type]         # Map from TVar.id to Type
Env   = Dict[str, Scheme]       # Γ: var -> scheme

def ftv_type(t: Type) -> Set[int]:
    if isinstance(t, TVar): return {t.id}
    if isinstance(t, TCon): return set()
    if isinstance(t, TFun): return ftv_type(t.arg) | ftv_type(t.ret)
    if isinstance(t, TPair):return ftv_type(t.fst) | ftv_type(t.snd)
    raise TypeError("unknown type")

def ftv_scheme(s: Scheme) -> Set[int]:
    return ftv_type(s.ty) - set(s.vars)

def ftv_env(gamma: Env) -> Set[int]:
    acc: Set[int] = set()
    for sch in gamma.values():
        acc |= ftv_scheme(sch)
    return acc

def apply_subst_type(s: Subst, t: Type) -> Type:
    if isinstance(t, TVar):
        return apply_subst_type(s, s[t.id]) if t.id in s else t
    if isinstance(t, TCon): return t
    if isinstance(t, TFun): return TFun(apply_subst_type(s, t.arg), apply_subst_type(s, t.ret))
    if isinstance(t, TPair):return TPair(apply_subst_type(s, t.fst), apply_subst_type(s, t.snd))
    raise TypeError("unknown type")

def apply_subst_scheme(s: Subst, sch: Scheme) -> Scheme:
    # Do not substitute bound variables
    s2 = {k:v for (k,v) in s.items() if k not in sch.vars}
    return Scheme(sch.vars, apply_subst_type(s2, sch.ty))

def apply_subst_env(s: Subst, gamma: Env) -> Env:
    return {x: apply_subst_scheme(s, sc) for (x, sc) in gamma.items()}

def compose(s2: Subst, s1: Subst) -> Subst:
    # (s2 ∘ s1) t = s2 ( s1 t )
    out = {k: apply_subst_type(s2, v) for (k, v) in s1.items()}
    out.update(s2)
    return out

class TVarGen:
    def __init__(self): self.cnt = 0
    def fresh(self) -> TVar:
        self.cnt += 1
        return TVar(self.cnt)

gen = TVarGen()

def instantiate(s: Scheme) -> Type:
    # Replace bound vars with fresh ones
    subst: Subst = {}
    for vid in s.vars:
        subst[vid] = gen.fresh()
    return apply_subst_type(subst, s.ty)

def generalize(gamma: Env, t: Type) -> Scheme:
    vars_to_bind = ftv_type(t) - ftv_env(gamma)
    return Scheme(frozenset(vars_to_bind), t)

In [ ]:
# =============================
# 🤝 Unification + Algorithm W
# =============================
class TypeErrorW(Exception): ...

def occurs_check(v: int, t: Type) -> bool:
    return v in ftv_type(t)

def var_bind(v: int, t: Type) -> Subst:
    if isinstance(t, TVar) and t.id == v: return {}
    if occurs_check(v, t): raise TypeErrorW("occurs check failed")
    return {v: t}

def unify(t1: Type, t2: Type) -> Subst:
    if isinstance(t1, TVar): return var_bind(t1.id, t2)
    if isinstance(t2, TVar): return var_bind(t2.id, t1)
    if isinstance(t1, TCon) and isinstance(t2, TCon):
        if t1.name != t2.name: raise TypeErrorW(f"type mismatch: {t1.name} vs {t2.name}")
        return {}
    if isinstance(t1, TFun) and isinstance(t2, TFun):
        s1 = unify(t1.arg, t2.arg)
        s2 = unify(apply_subst_type(s1, t1.ret), apply_subst_type(s1, t2.ret))
        return compose(s2, s1)
    if isinstance(t1, TPair) and isinstance(t2, TPair):
        s1 = unify(t1.fst, t2.fst)
        s2 = unify(apply_subst_type(s1, t1.snd), apply_subst_type(s1, t2.snd))
        return compose(s2, s1)
    raise TypeErrorW(f"cannot unify {show_type(t1)} with {show_type(t2)}")

In [ ]:
# =============================
# 🧩 HM expression language
# =============================
from dataclasses import dataclass

@dataclass(frozen=True) class Var:  x: str
@dataclass(frozen=True) class Lam:  x: str; body: "Expr"        # λx. e
@dataclass(frozen=True) class App:  f: "Expr"; a: "Expr"         # f a
@dataclass(frozen=True) class Let:  x: str; e1: "Expr"; e2: "Expr"
@dataclass(frozen=True) class IntLit:  n: int
@dataclass(frozen=True) class BoolLit: b: bool
@dataclass(frozen=True) class If:   c: "Expr"; t: "Expr"; e: "Expr"
@dataclass(frozen=True) class Add:  a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Eq:   a: "Expr"; b: "Expr"
@dataclass(frozen=True) class Pair: fst: "Expr"; snd: "Expr"

Expr = Var | Lam | App | Let | IntLit | BoolLit | If | Add | Eq | Pair

In [ ]:
# =============================
# 🧠 Algorithm W (principal types)
# =============================
def W(gamma: Env, e: Expr) -> Tuple[Subst, Type]:
    if isinstance(e, Var):
        if e.x not in gamma: raise TypeErrorW(f"unbound variable {e.x}")
        return {}, instantiate(gamma[e.x])
    if isinstance(e, IntLit):  return {}, INT
    if isinstance(e, BoolLit): return {}, BOOL
    if isinstance(e, Lam):
        a = gen.fresh()
        gamma2 = dict(gamma); gamma2[e.x] = Scheme(frozenset(), a)
        s1, t1 = W(gamma2, e.body)
        return s1, TFun(apply_subst_type(s1, a), t1)
    if isinstance(e, App):
        s1, t1 = W(gamma, e.f)
        s2, t2 = W(apply_subst_env(s1, gamma), e.a)
        a = gen.fresh()
        s3 = unify(apply_subst_type(s2, t1), TFun(t2, a))
        return compose(s3, compose(s2, s1)), apply_subst_type(s3, a)
    if isinstance(e, Let):
        s1, t1 = W(gamma, e.e1)
        sch = generalize(apply_subst_env(s1, gamma), t1)
        gamma2 = apply_subst_env(s1, gamma); gamma2[e.x] = sch
        s2, t2 = W(gamma2, e.e2)
        return compose(s2, s1), t2
    if isinstance(e, If):
        s1, tC = W(gamma, e.c)
        s1b = unify(tC, BOOL)
        gamma1 = apply_subst_env(s1b, apply_subst_env(s1, gamma))
        s2, tT = W(gamma1, e.t)
        gamma2 = apply_subst_env(s2, gamma1)
        s3, tE = W(gamma2, e.e)
        s4 = unify(apply_subst_type(s3, tT), tE)
        s_total = compose(s4, compose(s3, compose(s2, compose(s1b, s1))))
        return s_total, apply_subst_type(s4, tE)
    if isinstance(e, Add):
        s1, t1 = W(gamma, e.a)
        s2, t2 = W(apply_subst_env(s1, gamma), e.b)
        s3 = unify(apply_subst_type(s2, t1), INT)
        s4 = unify(apply_subst_type(s3, t2), INT)
        return compose(s4, compose(s3, compose(s2, s1))), INT
    if isinstance(e, Eq):
        s1, t1 = W(gamma, e.a)
        s2, t2 = W(apply_subst_env(s1, gamma), e.b)
        s3 = unify(apply_subst_type(s2, t1), t2)
        return compose(s3, compose(s2, s1)), BOOL
    if isinstance(e, Pair):
        s1, t1 = W(gamma, e.fst)
        s2, t2 = W(apply_subst_env(s1, gamma), e.snd)
        return compose(s2, s1), TPair(apply_subst_type(s2, t1), t2)
    raise TypeErrorW("unknown expression form")

In [ ]:
# =============================
# ✅ Self-checks (run to verify)
# =============================
def principal_type(e: Expr) -> str:
    s, t = W({}, e)
    return show_type(t)

# let id = \x. x in (id 3, id true)  ⇒  Int * Bool
id_fun = Lam("x", Var("x"))
prog1 = Let("id", id_fun, Pair(App(Var("id"), IntLit(3)), App(Var("id"), BoolLit(True))))
print("ok  - prog1:", principal_type(prog1))

# compose = \f.\g.\x. f (g x)  ⇒  (b -> c) -> (a -> b) -> a -> c
compose = Lam("f", Lam("g", Lam("x", App(Var("f"), App(Var("g"), Var("x"))))))
print("ok  - compose:", principal_type(compose))

# const = \a.\b. a;  (const 5 true) : Int
const = Lam("a", Lam("b", Var("a")))
print("ok  - const-app:", principal_type(App(App(const, IntLit(5)), BoolLit(True))))

# if (3==3) then 42 else 0 : Int
prog2 = If(Eq(IntLit(3), IntLit(3)), IntLit(42), IntLit(0))
print("ok  - prog2:", principal_type(prog2))

# occurs-check failure: \x. x x  (should raise)
try:
    bad = Lam("x", App(Var("x"), Var("x")))
    principal_type(bad)
    print("FAIL- occurs-check: expected failure but succeeded")
except Exception as ex:
    print("ok  - occurs-check raised:", type(ex).__name__)

### 🧪 Your Turn (10–15 minutes)
1. Define `pairId = \p. p` and infer its type; explain in 1–2 sentences why it generalizes to `α * β -> α * β`.  
2. Extend the language with `Fst` and `Snd` expressions and write their typing rules (no code required):  
   `Fst : (α * β) -> α`, `Snd : (α * β) -> β`.  
3. (Reasoning) If we had a subtyping relation `Int <: Num`, which side of `τ1 -> τ2` would be **contravariant** and which **covariant**? Give one sentence that justifies your answer.

### ✍️ Reflection (2–3 sentences)
- Where do **generalization** and **instantiation** appear in Algorithm W, and why are both necessary for `let`‑polymorphism?

In [ ]:
# 📝 Save small submission bundle
import json, time
stamp = time.strftime("%Y-%m-%d %H:%M:%S")
submission = {
    "student_name": STUDENT_NAME,
    "student_id": STUDENT_ID,
    "timestamp": stamp,
    "checks": ["prog1", "compose", "const-app", "prog2", "occurs-check"],
    "reflection": "(fill in here)"
}
with open("week5_submission.json", "w") as f:
    json.dump(submission, f, indent=2)
print("Saved week5_submission.json — upload with your notebook.")